<a href="https://colab.research.google.com/github/zhr-frj/daftarchePhd/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Wed Nov 12 12:56:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# --- سلول ۱: نصب استاندارد و پایدار ---
print("--- در حال نصب کتابخانه‌های استاندارد ---")
!pip install langchain langchain-community transformers bitsandbytes accelerate faiss-cpu pypdf sentence-transformers langchain_huggingface docarray -q
print("--- نصب کامل شد ---")

--- در حال نصب کتابخانه‌های استاندارد ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
--- نصب کامل شد ---


In [4]:
# --- وارد کردن ابزارها و تنظیمات فشرده‌سازی ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import BitsAndBytesConfig
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser

print("--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---")
# تنظیمات فشرده‌سازی 4-bit برای کاهش مصرف حافظه
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_quant_type="nf4"
)
print("--- ابزارها و تنظیمات آماده شدند ---")

--- در حال آماده‌سازی تنظیمات فشرده‌سازی (Quantization) ---
--- ابزارها و تنظیمات آماده شدند ---


In [5]:
# # --- بارگذاری «مترجم» (Tokenizer) ---
# print("--- در حال بارگذاری «مترجم» (Tokenizer) ---")
# tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-9b-it", use_fast=True)
# print("--- «مترجم» (Tokenizer) بارگذاری شد ---")


# --- سلول ۲.۲: بارگذاری «مترجم» (Tokenizer) - نسخه سبک Gemma 2B ---
from transformers import AutoTokenizer

print("--- در حال بارگذاری «مترجم» (Gemma 2B) ---")
tokenizer = AutoTokenizer.from_pretrained(
    "unsloth/gemma-2-2b-it",
    use_fast=True
    )
print("--- «مترجم» (Tokenizer) بارگذاری شد ---")

--- در حال بارگذاری «مترجم» (Gemma 2B) ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

--- «مترجم» (Tokenizer) بارگذاری شد ---


In [6]:
# # --- بارگذاری «مغز» (LLM) ---
# print("--- در حال بارگذاری «مغز» (LLM) --- (این مرحله بسیار سنگین و زمان‌بر است)")
# model = AutoModelForCausalLM.from_pretrained(
#     "unsloth/gemma-2-9b-it",
#     device_map="auto",
#     quantization_config=bnb_config,
#     torch_dtype=torch.float16
# )
# print("--- «مغز» (LLM) با موفقیت بارگذاری شد ---")



# --- سلول ۲.۳: بارگذاری «مغز» (LLM) - نسخه سبک Gemma 2B ---
from transformers import AutoModelForCausalLM

print("--- در حال بارگذاری «مغز» (LLM) --- (نسخه سبک Gemma 2B)")
model = AutoModelForCausalLM.from_pretrained(
    "unsloth/gemma-2-2b-it",
    device_map="auto",
    quantization_config=bnb_config, # از bnb_config استفاده می‌کند
    torch_dtype=torch.float16
)
print("--- «مغز» (LLM) با موفقیت بارگذاری شد ---")

--- در حال بارگذاری «مغز» (LLM) --- (نسخه سبک Gemma 2B)


config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

--- «مغز» (LLM) با موفقیت بارگذاری شد ---


In [7]:
# --- ساخت «خط لوله» و ابزارهای LangChain ---
print("--- در حال ساخت «خط لوله» (Pipeline) ---")
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512  # افزایش طول پاسخ برای جواب‌های کامل‌تر
)

# ساخت «غلاف» LangChain برای LLM
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# ساخت «تمیزکننده خروجی»
parser = StrOutputParser()

print("--- «خط لوله»، «غلاف» و «تمیزکننده» آماده شدند ---")

Device set to use cuda:0


--- در حال ساخت «خط لوله» (Pipeline) ---
--- «خط لوله»، «غلاف» و «تمیزکننده» آماده شدند ---


In [8]:
# --- تابع کمکی برای اعمال «قالب چت» ---

# این تابع، پرامپت نهایی ما را در قالب چت رسمی مدل Gemma قرار می‌دهد
def apply_chat_template_and_response(prompt):
    messages = [
    {'role': 'user', 'content': prompt}
    ]

    # اعمال قالب چت
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    # فراخوانی مدل و تمیزکاری اولیه
    response = llm.invoke(text)
    # response را تمیز می‌کنیم تا فقط جواب نهایی باقی بماند
    return response.replace(text, '').strip()

print("--- تابع کمکی «قالب چت» آماده شد ---")

--- تابع کمکی «قالب چت» آماده شد ---


In [19]:
# --- سلول ۴: بارگذاری «بینی معنی‌فهم» (Embedding Model) ---

from langchain_huggingface import HuggingFaceEmbeddings

print("--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) ---")

# --- تغییر کلیدی ---
# به جای 'sbunlp/fabert' از این مدل قوی‌تر استفاده می‌کنیم
embedding_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# --- پایان تغییر ---

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_name,
    model_kwargs={'device': 'cpu'} # همچنان روی CPU اجرا می‌کنیم تا GPU آزاد بماند
)
print("--- «بینی معنی‌فهم» (Embeddings) آماده شد ---")

--- در حال بارگذاری «بینی معنی‌فهم» (Embeddings) ---


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

--- «بینی معنی‌فهم» (Embeddings) آماده شد ---


In [20]:
# # --- تعریف «برگه دستورالعمل» (Prompt Template) ---

# from langchain.prompts import PromptTemplate

# # این الگو به ربات می‌گوید که «خارج از PDF» جواب ندهد
# template = """
# شما یک دستیار هوشمند و بسیار دقیق هستید. وظیفه شما پاسخ به سوال کاربر فقط و فقط بر اساس متنی است که در بخش «زمینه» به شما داده می‌شود.
# به هیچ وجه از دانش قبلی خود استفاده نکنید.
# پاسخ را فقط به زبان فارسی ارائه دهید.
# اگر پاسخ سوال در «زمینه» وجود نداشت، به سادگی بگویید: "متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم."

# زمینه (متن‌های یافت‌شده از PDF):
# {context}

# سوال کاربر:
# {question}

# پاسخ دقیق شما:
# """

# prompt = PromptTemplate.from_template(template)
# print("--- «برگه دستورالعمل» (Prompt) آماده شد ---")





# --- سلول ۵: تعریف «برگه دستورالعمل» (Prompt Template) - نسخه Gemma ---
from langchain.prompts import PromptTemplate

# قالب پرامپت استاندارد (بدون تگ‌های <|user|>)
template = """
شما یک دستیار هوشمند و بسیار دقیق هستید. وظیفه شما پاسخ به سوال کاربر فقط و فقط بر اساس متنی است که در بخش «زمینه» به شما داده می‌شود.
به هیچ وجه از دانش قبلی خود استفاده نکنید.
پاسخ را فقط به زبان فارسی ارائه دهید.
اگر پاسخ سوال در «زمینه» وجود نداشت، به سادگی بگویید: "متاسفانه اطلاعاتی در این مورد در اسناد پیدا نکردم."

زمینه (متن‌های یافت‌شده از PDF):
{context}

سوال کاربر:
{question}

پاسخ دقیق شما:
"""

prompt = PromptTemplate.from_template(template)
print("--- «برگه دستورالعمل» (Prompt) مخصوص Gemma آماده شد ---")

--- «برگه دستورالعمل» (Prompt) مخصوص Gemma آماده شد ---


In [11]:
# --- دانلود فایل‌های PDF ---
print("--- در حال دانلود فایل‌های PDF ---")
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Phd1405-.konkur.in.pdf" -O daftarche_asli.pdf
!wget -q "https://github.com/zhr-frj/daftarchePhd/releases/download/v1.0.0/Eslahiye-PHD1405-.konkur.in.pdf" -O daftarche_eslahiye.pdf
print("--- فایل‌ها دانلود شدند ---")

--- در حال دانلود فایل‌های PDF ---
--- فایل‌ها دانلود شدند ---


In [12]:
# --- خواندن، بارگذاری و خُرد کردن متن‌ها ---
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ۱. خواندن هر دو فایل PDF
print("... در حال خواندن فایل‌های PDF")
loader_asli = PyPDFLoader("daftarche_asli.pdf")
loader_eslahiye = PyPDFLoader("daftarche_eslahiye.pdf")
all_pages = loader_asli.load() + loader_eslahiye.load()
print(f"--- مجموعا {len(all_pages)} صفحه بارگذاری شد ---")

# ۲. خُرد کردن متن‌ها به تکه‌های کوچک (Chunks)
print("... در حال خُرد کردن متن‌ها")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
text_documents = text_splitter.split_documents(all_pages)
print(f"--- متن‌ها به {len(text_documents)} تکه (Chunk) تقسیم شدند ---")

... در حال خواندن فایل‌های PDF
--- مجموعا 58 صفحه بارگذاری شد ---
... در حال خُرد کردن متن‌ها
--- متن‌ها به 206 تکه (Chunk) تقسیم شدند ---


In [13]:
# --- ساخت «بایگانی هوشمند» و «کتابدار» ---
# from langchain_community.vectorstores import DocArrayInMemorySearch

# # ۱. ساخت «بایگانی هوشمند» (Vectorstore)
# # (این مرحله تمام تکه‌ها را به وکتور تبدیل می‌کند و سنگین‌ترین بخش است)
# print("... در حال ساخت «بایگانی هوشمند» (Vectorstore)... (این مرحله زمان‌بر است)")
# vectorstore = DocArrayInMemorySearch.from_documents(
#     text_documents,
#     embedding=embeddings # از متغیر embeddings که در سلول‌های قبلی ساختیم استفاده می‌کند
# )
# print("--- «بایگانی هوشمند» با موفقیت ساخته شد ---")

# # ۲. ساخت «کتابدار» (Retriever)
# retriever = vectorstore.as_retriever(search_kwargs={'k': 3}) # ۳ سند مرتبط را پیدا کن
# print("--- «کتابدار» (Retriever) آماده جستجو است ---")




# --- سلول ۶.۳ (الف): ساخت یا بارگذاری «بایگانی هوشمند» ---
from langchain_community.vectorstores import FAISS
import os

DB_FAISS_PATH = "faiss_index_daftarche" # نام پوشه‌ای که بایگانی در آن ذخیره می‌شود

# حذف بایگانی قدیمی (چون مدل امبدینگ عوض شده)
!rm -rf faiss_index_daftarche

if not os.path.exists(DB_FAISS_PATH):
    print("... «بایگانی هوشمند» (Vectorstore) وجود ندارد. در حال ساخت...")
    # (این مرحله تمام تکه‌ها را به وکتور تبدیل می‌کند و سنگین‌ترین بخش است)
    vectorstore = FAISS.from_documents(
        text_documents,
        embedding=embeddings # از متغیر embeddings قبلی استفاده می‌کند
    )

    # --- ذخیره کردن «مغز» روی دیسک ---
    vectorstore.save_local(DB_FAISS_PATH)
    print(f"--- «بایگانی هوشمند» با موفقیت ساخته و در {DB_FAISS_PATH} ذخیره شد ---")

else:
    print(f"--- «بایگانی هوشمند» از قبل در {DB_FAISS_PATH} وجود دارد. در حال بارگذاری...")
    # --- بارگذاری «مغز» از دیسک (بسیار سریع) ---
    vectorstore = FAISS.load_local(DB_FAISS_PATH, embeddings, allow_dangerous_deserialization=True)
    print("--- «بایگانی هوشمند» با موفقیت بارگذاری شد ---")

... «بایگانی هوشمند» (Vectorstore) وجود ندارد. در حال ساخت...
--- «بایگانی هوشمند» با موفقیت ساخته و در faiss_index_daftarche ذخیره شد ---


In [14]:
# --- سلول ۶.۳ (ب): ساخت «کتابدار» (Retriever) ---
# (این سلول باید بعد از سلول ۶.۳ (الف) اجرا شود)

retriever = vectorstore.as_retriever(search_kwargs={'k': 3}) # ۳ سند مرتبط را پیدا کن
print("--- «کتابدار» (Retriever) آماده جستجو است ---")

--- «کتابدار» (Retriever) آماده جستجو است ---


In [15]:
# --- تعریف تابع نهایی پرسش و پاسخ ---

print("--- در حال تعریف تابع 'ask_robot' ---")

def ask_robot(question):
    """
    این تابع یک سوال می‌گیرد و تمام مراحل RAG را انجام می‌دهد.
    """
    try:
        print("... 🔎 در حال جستجو در دفترچه‌ها ...")

        # ۱. پیدا کردن تکه‌های متن مرتبط (Retrieve)
        retrieved_context_docs = retriever.invoke(question)
        # تبدیل لیست اسناد به یک متن واحد
        retrieved_context = "\n\n".join(doc.page_content for doc in retrieved_context_docs)

        print("... 🧠 در حال آماده‌سازی پاسخ ...")

        # ۲. ساخت پرامپت نهایی با «زمینه» و «سوال» (Augment)
        formatted_prompt = prompt.format(context=retrieved_context, question=question)

        # ۳. ارسال پرامپت نهایی به مدل برای تولید جواب (Generate)
        response_from_model = apply_chat_template_and_response(formatted_prompt)

        # ۴. تمیزکاری و نمایش جواب
        parsed_response = parser.parse(response_from_model)
        return parsed_response

    except Exception as e:
        return f"خطایی در پردازش رخ داد: {e}"

print("--- تابع 'ask_robot' با موفقیت ساخته شد و آماده است ---")

--- در حال تعریف تابع 'ask_robot' ---
--- تابع 'ask_robot' با موفقیت ساخته شد و آماده است ---


In [18]:
# --- سلول ۸ (نهایی): اجرای تمام سوالات تستی و فنی ---

print("--- 🤖 شروع تست ربات با سوالات حرفه‌ای 🤖 ---")

# --- دسته ۱: سوالات اصلی شما ---

query_1 = "سهمیه ایثارگران چقدر است؟"
print(f"[شما]: {query_1}")
answer_1 = ask_robot(query_1)
print(f"\n[ربات]:\n{answer_1}")
print("\n" + "="*50)

query_2 = "گرایش های رشته زبان و ادبیات فارسی؟"
print(f"[شما]: {query_2}")
answer_2 = ask_robot(query_2)
print(f"\n[ربات]:\n{answer_2}")
print("\n" + "="*50)

query_3 = "گرایش های علوم اقتصادي؟"
print(f"[شما]: {query_3}")
answer_3 = ask_robot(query_3)
print(f"\n[ربات]:\n{answer_3}")
print("\n" + "="*50)

# --- دسته ۲: سوالات فنی و دقیق (برای نمایش قدرت جستجو) ---

query_4 = "نمره کل نهایی چگونه محاسبه می شود؟"
print(f"[شما]: {query_4}")
answer_4 = ask_robot(query_4)
print(f"\n[ربات]:\n{answer_4}")
print("\n" + "="*50)

query_5 = "ضریب تاثیر سوابق آموزشی، پژوهشی و فناوری چند درصد است؟"
print(f"[شما]: {query_5}")
answer_5 = ask_robot(query_5)
print(f"\n[ربات]:\n{answer_5}")
print("\n" + "="*50)

# --- دسته ۳: سوالات ترکیبی (برای نمایش درک اصلاحیه) ---

query_6 = "کد رشته محل‌های مربوط به رشته مهندسی کامپیوتر - هوش مصنوعی کدامند؟"
print(f"[شما]: {query_6}")
answer_6 = ask_robot(query_6)
print(f"\n[ربات]:\n{answer_6}")
print("\n" + "="*50)

query_7 = "بر اساس اصلاحیه، تاریخ دقیق بررسی سوابق علمی و مصاحبه چه زمانی است؟"
print(f"[شما]: {query_7}")
answer_7 = ask_robot(query_7)
print(f"\n[ربات]:\n{answer_7}")
print("\n" + "="*50)

# --- دسته ۴: سوالات کنترلی (برای نمایش ایمنی ربات) ---

query_8 = "اسم من چیست؟"
print(f"[شما]: {query_8}")
answer_8 = ask_robot(query_8)
print(f"\n[ربات]:\n{answer_8}")
print("\n" + "="*50)

query_9 = "پایتخت ایران کجاست؟"
print(f"[شما]: {query_9}")
answer_9 = ask_robot(query_9)
print(f"\n[ربات]:\n{answer_9}")
print("\n" + "="*50)

print("--- 🏁 تست ربات تمام شد 🏁 ---")

--- 🤖 شروع تست ربات با سوالات حرفه‌ای 🤖 ---
[شما]: سهمیه ایثارگران چقدر است؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
سهمیه ایثارگران،  از  ۵۲ درصد تا ۵ درصد از ظرفیت هر کدرشتهمحل به  «همسر و فرزندانشهدا و مفقودالاثران»،«آزادگان و همسر و فرزندان آنان»،«جانبازان۵۲% و بالاتر و همسر و فرزندان آنان»  اختصاص دارد.

[شما]: گرایش های رشته زبان و ادبیات فارسی؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



[ربات]:
رشته گرايش 
کد 
ترکيب  
ترکيب دروس امتحاني بر اساس شماره دروس 
1 2 3  
 آموزش زبان و ادبيات فارسی - 1   
رشتههاي مرتبط که فارغالتحصيلان آن ميتوانند در اين مجموعه امتحاني شرکت کنند: 
زبان و ادبيات فارسی

[شما]: گرایش های علوم اقتصادي؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
1) اقتصادسنجي 
2) اقتصاد پولي 
3) اقتصاد اسلامي 
4) اقتصاد بخش عمومي 
5) توسعه اقتصادي 
6) اقتصاد بینالملل 
7) اقتصاد مالي 
8) اقتصاد شهري و منطقهاي 
9) اقتصاد منابع 
10) اقتصاد بيمه 
11) اقتصاد نهادگرا 
12) اقتصاد رياضي 
13) اقتصاد ايران

[شما]: نمره کل نهایی چگونه محاسبه می شود؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
متاسفانه اطلاعاتی در این مورد در اسناد یافت نشد.

[شما]: ضریب تاثیر سوابق آموزشی، پژوهشی و فناوری چند درصد است؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در حال آماده‌سازی پاسخ ...

[ربات]:
30 درصد

[شما]: کد رشته محل‌های مربوط به رشته مهندسی کامپیوتر - هوش مصنوعی کدامند؟
... 🔎 در حال جستجو در دفترچه‌ها ...
... 🧠 در